In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, auc,roc_auc_score, precision_score, recall_score,f1_score, roc_curve
from matplotlib import pyplot as plt
from collections import Counter

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# **Import data**

In [2]:
data_path = 'https://raw.githubusercontent.com/sahdan96/randomforest/main/train_2v.csv'
data = pd.read_csv(data_path)

# **pre-processing data**

In [3]:
data['bmi'] = data['bmi'].fillna(data['bmi'].mean())
data.dropna(axis=0, inplace=True)
data.drop(columns ='id', inplace =True)

In [4]:
encode_gender = LabelEncoder()
encode_marry = LabelEncoder()
encode_work = LabelEncoder()
encode_residence = LabelEncoder()
encode_smoking = LabelEncoder()
data['gender'] = encode_gender.fit_transform(data['gender'])
data['ever_married'] = encode_marry.fit_transform(data['ever_married'])
data['work_type'] = encode_work.fit_transform(data['work_type'])
data['Residence_type'] = encode_residence.fit_transform(data['Residence_type'])
data['smoking_status'] = encode_smoking.fit_transform(data['smoking_status'])

# **split data**

In [5]:
x = data.drop('stroke', axis=1)
y = data.stroke

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [7]:
print("train class count: \n" +str(y_train.value_counts())+"\n\ntest class count: \n" + str(y_test.value_counts()))

train class count: 
0    23573
1      513
Name: stroke, dtype: int64

test class count: 
0    5897
1     125
Name: stroke, dtype: int64


# **Oversampling**

In [8]:
ros = SMOTE(random_state=0)
X_train_resampled, Y_train_resampled = ros.fit_sample(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [9]:
## train class count
print("before sampling:\n"+ str(Counter(y_train))+"\n\nafter sampling:\n"+ str(Counter(Y_train_resampled)))

before sampling:
Counter({0: 23573, 1: 513})

after sampling:
Counter({0: 23573, 1: 23573})


# **training + testing/prediction**

In [10]:
rf_model = RandomForestClassifier(bootstrap=True, max_depth=None, 
                                  max_features='auto', max_leaf_nodes= None,
                                  min_impurity_decrease= 0.0, min_impurity_split=None,
                                  min_samples_leaf= 1, min_samples_split= 2,
                                  min_weight_fraction_leaf=0.0, 
                                  n_estimators=10, random_state=0,
                                n_jobs=None, oob_score=False,
                                  verbose=0, warm_start=False)

rf_model.fit(X_train_resampled, Y_train_resampled)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [11]:
y_train_pred = rf_model.predict(X_train_resampled)
y_test_pred = rf_model.predict(X_test)

# **train & test benchmark**

In [12]:
acc = accuracy_score(Y_train_resampled, y_train_pred)
pre = precision_score(Y_train_resampled, y_train_pred)
re = recall_score(Y_train_resampled, y_train_pred)
f1 = f1_score(Y_train_resampled, y_train_pred)

acc2 = accuracy_score(y_test, y_test_pred)
pre2 = precision_score(y_test, y_test_pred)
re2 = recall_score(y_test, y_test_pred)
f12 = f1_score(y_test, y_test_pred)
print("training benchmark:")
print("accuracy:\t"+str(acc)+"\nprecision:\t"+str(pre)+"\nrecall:\t\t"+str(re)+"\nf1:\t\t"+str(f1))
print("\ntesting benchmark:")
print("accuracy:\t"+str(acc2)+"\nprecision:\t"+str(pre2)+"\nrecall:\t\t"+str(re2)+"\nf1:\t\t"+str(f12))

training benchmark:
accuracy:	0.9979001399906673
precision:	0.9999574033055035
recall:		0.9958427013956646
f1:		0.9978958107504942

testing benchmark:
accuracy:	0.9775821986051145
precision:	0.14285714285714285
recall:		0.016
f1:		0.02877697841726618


In [13]:
cm= confusion_matrix(Y_train_resampled,y_train_pred)
cm2= confusion_matrix(y_test,y_test_pred)
print("confusion matrix for training:\n"+str(cm)+"\n\nconfusion matrix for testing:\n"+str(cm2))

confusion matrix for training:
[[23572     1]
 [   98 23475]]

confusion matrix for testing:
[[5885   12]
 [ 123    2]]


In [ ]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5887
           1       0.19      0.03      0.05       135

    accuracy                           0.98      6022
   macro avg       0.58      0.51      0.52      6022
weighted avg       0.96      0.98      0.97      6022

